# INSTRUCTIONS FOR IMAGE SPACE SETUP

Step 1 - Download 80k images:
1. Open file download_images.ipynb and run the code from top to bottom. 80,000+ images will begin to download in the 95k_Images folder.
Use ls -1 | wc -l in 95k_Images to view download progress.

Step 2 - Create a folder subset of 1,000 Images:
1. /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
2. brew install coreutils
3. Run this on 95k_Images to get a subset: find 95k_Images/ -type f -print0 | shuf -n 1000 -z | xargs -0 -I{} cp {} subset_images/

Step 3 - Download Image Space repo:
1. run this on 8_ImageSpace Folder: git clone https://github.com/nasa-jpl-memex/image_space.git

Step 4 - Set environment variables:
1. export IMAGE_SPACE=/Users/toddgavin/Desktop/USC_Classes/DSCI550-Data_Science_at_Scale/Assignment_3/GitHub/DSCI550_PixstoryDatasetAnalysisVisualizations/8_ImageSpace/image_space
    - verify path: echo $IMAGE_SPACE 
2. export IMAGE_DIR=/Users/toddgavin/Desktop/USC_Classes/DSCI550-Data_Science_at_Scale/Assignment_3/GitHub/DSCI550_PixstoryDatasetAnalysisVisualizations/8_ImageSpace/subset_images
    - verify path: echo $IMAGE_DIR

Step 5 - Build your Neural Net (AlexNet) based features for the images.
1. cd "$IMAGE_SPACE/imagespace_smqtk" && ./smqtk_services.run_images.sh --docker-network deploy_imagespace-network --images "$IMAGE_DIR"
    - If having trouble, try to tear down the old network "docker network prune"
    - you should see "docker ps -a" showing NOTHING
    - "docker network ls --no-trunc" showing NOTHING






Old stuff:

2. Run this on image_space folder: cd /imagespace_smqtk && ./smqtk_services.run_images.sh --docker-network deploy_imagespace-network --images ../../95k_Images_v2
3. Run this on image_space folder: cd "scripts/deploy" && IMAGE_DIR=../../../95k_Images_v2 docker-compose up -d
4. cd "../../scripts/deploy" && sh ./imagespace/enable-imagespace.sh

I ran these two commands and #4 seemed to work:
docker rename deploy-imagespace-mongo-1 deploy_imagespace-mongo_1
docker rename deploy-imagespace-girder-1 deploy_imagespace-girder_1

this works: docker exec -it deploy-imagespace-mongo-1 mongo girder --eval 'db.setting.update({key: "core.plugins_enabled"}, {$push: {value: "imagespace_smqtk"}})'
this works: docker exec -it deploy-imagespace-mongo-1 mongo girder --eval 'db.setting.insert({key: "IMAGE_SPACE_SMQTK_NNSS_URL", value: "http://smqtk-services:12345"})'
this works: docker exec -it deploy-imagespace-mongo-1 mongo girder --eval 'db.setting.insert({key: "IMAGE_SPACE_SMQTK_IQR_URL", value: "http://smqtk-services:12346"})'
this works: docker exec -it deploy-imagespace-mongo-1 mongo girder --eval 'db.setting.insert({key: "IMAGE_SPACE_DEFAULT_SIMILARITY_SEARCH", value: "smqtk-similarity"})'
??: docker exec -it deploy-imagespace-mongo-1 touch /girder/girder/conf/girder.dist.cfg


- Ingest a subset of your Pixstory 95k images / posts into Image Space using the provided instructions and scripts or the ones you write on your own using Tika-Python.
- Browse and find similar images and use the ImageSpace search index and search the Image forensics and similarity (SMQTK).    